# Co-Factor Swap

Uses co-factor swaps to optimize production of a target metabolite.

Reference reading 
[https://doi.org/10.1016/j.ymben.2014.05.009](https://doi.org/10.1016/j.ymben.2014.05.009).


In [1]:
from cobra.io import read_sbml_model
from reframed.io.sbml import load_cbmodel
from mewpy.problems import CofactorSwapProblem
from mewpy.optimization.evaluation import TargetFlux, BPCY
from mewpy.optimization import EA
from mewpy import get_simulator

In [2]:
model = read_sbml_model('../models/ec/iJO1366.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-30


In [3]:
for rid in ['FHL', 'CAT', 'SPODM', 'SPODMpp']:
    model.reactions.get_by_id(rid).knock_out()
model.reactions.POR5.lower_bound = 0
model.reactions.EX_glc__D_e.lower_bound = -10
model.reactions.EX_o2_e.lower_bound = -10

In [4]:
BIOMASS = 'BIOMASS_Ec_iJO1366_core_53p95M'
PRODUCT = 'EX_thr__L_e'

In [5]:
objectives = [TargetFlux(PRODUCT)]

problem = CofactorSwapProblem(model,objectives)

problem.simulator.set_reaction_bounds(BIOMASS,0.1,1000)
problem.simulator.objective=PRODUCT


Read LP format model from file /var/folders/fw/kbs61_l15j587pjbwf3_y8780000gn/T/tmpml814svu.lp
Reading time = 0.01 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [6]:
ea = EA(problem,max_generations=10)
final_population = ea.run()

Running GA
Evaluations: 100	Fitness: 11.545761829545475
Evaluations: 200	Fitness: 12.082439942995249
Evaluations: 300	Fitness: 12.082439942995249
Evaluations: 400	Fitness: 12.08243994299525
Evaluations: 500	Fitness: 12.08243994299525
Evaluations: 600	Fitness: 12.082439942995268
Evaluations: 700	Fitness: 12.082439942995272
Evaluations: 800	Fitness: 12.082439942995272
Evaluations: 900	Fitness: 12.082439942995272
Evaluations: 1000	Fitness: 12.082439942995272


In [7]:
ea.dataframe()

,Modification,Size,TargetFlux
0,{'TRSARr': 0},1,12.08244


In [8]:
constraints = final_population[0].constraints
res = problem.simulate(constraints=constraints)

In [9]:
res.find([BIOMASS,PRODUCT,'glc'])

,Flux rate
Reaction ID,
EX_glc__D_e,-10.00000
BIOMASS_Ec_iJO1366_core_53p95M,0.10000
EX_thr__L_e,12.08244
